General approach:
1. For each PK cell in the table, get the top 3 entities from Wikidata.
2. For each other cell in the table, get the relations and objects for each of the top 3 entities.
3. For each column, get the most common relation.
4. For each column, get the object that has the most common relation with the PK cell.
5. For the PK column, get the most common type.

The code is a bit messy with a lot of loop, however, it does work. The results are fine, but there sure are improvements to be made.

In [101]:
import pandas as pd
import os
from lookup import WikidataAPI
from endpoints import WikidataEndpoint

In [102]:
# Query to return matches for object and relation given value from a QID
def get_relation_object_query(QID, value):
    if isinstance(value, str):
        filter_string = f'FILTER(?object = "{value}").'
    else:
        filter_string = f'FILTER(?object = {value}).'
    
    query = f"""
    SELECT ?relation ?object
    WHERE {{
      {{
        wd:{QID} ?relation ?object.
        ?object rdfs:label "{value}"@en.
      }}
      UNION
      {{
        wd:{QID} ?relation ?object.
        {filter_string}
      }}
    }}
    """
    return query


class CSV2KG:
    def __init__(self, df):
        self.wikidata = WikidataAPI()
        self.ep = WikidataEndpoint()
        self.df = df
        self.df_candidates = None
        self.df_relations = None
        self.df_entities = None
        self.df_classes = None
        
    def get_candidates(self):
        dataframe = pd.DataFrame(columns=self.df.columns, dtype=object)#self.df.copy()
        
        for idx, row in self.df.iterrows():
            search_string = row.iloc[0]
            
            # Entities
            entities = self.wikidata.getKGEntities(query=search_string, 
                                      limit=3, 
                                      type="item")
            
            dataframe.at[idx, self.df.columns[0]] = entities
            
                
            for search_idx in row.index[1:]:
                search_obj_dict = {}
                search_obj = row[search_idx]
                
                if search_obj == search_obj:  # Check if it is not NaN
                
                    for entity in entities:
                        QID = entity.getId().split("/")[-1]
                        
                        query = get_relation_object_query(QID, search_obj)
                        results = self.ep.getQueryResults(query)
                        
                        rel_obj_list = []
                        if len(results["results"]["bindings"]) > 0:
                            for res in results["results"]["bindings"]:
                                rel = res["relation"]["value"]
                                obj = res["object"]["value"]
                                rel_obj_list.append((rel, obj))
                                
                        search_obj_dict[entity.getId()] = rel_obj_list

                dataframe.at[idx, search_idx] = search_obj_dict
        
        self.df_candidates = dataframe
    
    
    def get_relations(self):
        counts = {}
        for col in self.df_candidates.columns[1:]:
            relation_counts = {}
            for idx, row in self.df_candidates.iterrows():
                for entity, search_obj_dict in row[col].items():
                    for rel, obj in search_obj_dict:
                        relation_counts[rel] = relation_counts.get(rel, 0) + 1
            counts[col] = relation_counts
            
        self.df_relations = pd.DataFrame(columns=self.df.columns)
        for col, rel_counts in counts.items():
            if rel_counts:
                relation = max(rel_counts, key=rel_counts.get)        
                self.df_relations.at[0, col] = relation
            else:
                self.df_relations.at[0, col] = None
            
    def get_entities(self):
        self.df_entities = pd.DataFrame(columns=self.df.columns, index=self.df.index)
        for idx, row in self.df_candidates.iterrows():
            for col in self.df_candidates.columns[1:]:
                for entity, search_obj_dict in row[col].items():
                    if self.df_relations.at[0, col] in [rel for rel, obj in search_obj_dict]:
                        if self.df_entities.at[idx, self.df_entities.columns[0]] != self.df_entities.at[idx, self.df_entities.columns[0]]:
                            self.df_entities.at[idx, self.df_entities.columns[0]] = entity
        
        for idx, row in self.df_entities.iterrows():
            for col in self.df_entities.columns[1:]:
                subject = self.df_entities.at[idx, self.df_entities.columns[0]]
                
                if subject == subject:
                    for prop, obj in self.df_candidates.at[idx, col].get(subject, []):
                        if prop == self.df_relations.at[0, col]:
                            self.df_entities.at[idx, col] = obj
                            
    def get_types(self):
        self.df_classes = pd.DataFrame(columns=self.df.columns)
        
        types_dct = {}
        for idx, row in self.df_candidates.iterrows():
            entites = row.iloc[0]
            for entity in entites:
                types = self.ep.getTypesForEntity(entity.getId())
                for t in types:
                    types_dct[t] = types_dct.get(t, 0) + 1
        
        if types_dct:
            type_entity = max(types_dct, key=types_dct.get)
        else:
            type_entity = None
        self.df_classes.at[0, self.df_classes.columns[0]] = type_entity
        
        for col in self.df_entities.columns[1:]:
            types_dct = {}
            for idx, row in self.df_entities.iterrows():
                entity = row[col]
                if entity == entity:
                    types = self.ep.getTypesForEntity(entity.split("/")[-1])
                    for t in types:
                        types_dct[t] = types_dct.get(t, 0) + 1
            
            if types_dct:
                type_entity = max(types_dct, key=types_dct.get)
                self.df_classes.at[0, col] = type_entity    
        
    
    def run_alignment(self):
        self.get_candidates()
        self.get_relations()
        self.get_entities()
        self.get_types()


# Run for 5 tables

In [103]:
# Download the data from https://github.com/sem-tab-challenge/2024/blob/main/data/WikidataTables2024R1.tar.gz
path_to_tables = './data/WikidataTables2024R1/DataSets/Valid/tables/'
file_names = os.listdir(path_to_tables)

In [104]:
results = []

for file_name in file_names[:10]:
    df = pd.read_csv(path_to_tables + file_name)
    mappings = CSV2KG(df)
    mappings.run_alignment()
    results.append([file_name, mappings])
    print(file_name)

03ENJ6XM.csv
08IZY6G9.csv
0CZW7M0F.csv
0DYQDSCY.csv
0ER7T81U.csv
0JGLUKC7.csv
0JLTIHOL.csv
0LBWW5L4.csv
0N0NEYH9.csv
0N5SQTLA.csv


# CEA (Cell-Entity Annotation)

In [105]:
cea = pd.read_csv("./data/WikidataTables2024R1/DataSets/Valid/gt/cea_gt.csv", header=None)

In [106]:
avg_precision = 0
avg_recall = 0
avg_f1 = 0

for res_no in range(len(results)):
    # These variables are used to calculate the precision, recall and F1 score
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    subset_df = cea[cea[0] == results[res_no][0].removesuffix('.csv')]
    for idx, row in subset_df.iterrows():
        mapping = results[res_no][1].df_entities.iloc[row[1] - 1, row[2]]
        if mapping == row[3]:  # If we found the correct mapping, increment the true positives
            true_positives += 1
        else:
            if mapping != mapping:  # If we didn't find a mapping, increment the false negatives
                false_negatives += 1
            else:  # If we found a mapping, but it was incorrect, increment the false positives
                false_positives += 1

    print(f"{results[res_no][0]}\n  Correct: {true_positives}, Incorrect: {false_negatives + false_positives}")

    try:
        precision = true_positives / (true_positives + false_positives)
    except ZeroDivisionError:
        precision = 0
    avg_precision += precision

    try:
        recall = true_positives / (true_positives + false_negatives)
    except ZeroDivisionError:
        recall = 0
    avg_recall += recall

    try:
        f1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0
    avg_f1 += f1
    
    print(f"  Precision: {precision},  Recall: {recall},  F1: {f1}")

print(f"Average Precision: {avg_precision / len(results)}, Average Recall: {avg_recall / len(results)}, Average F1: {avg_f1 / len(results)}")

03ENJ6XM.csv
  Correct: 2, Incorrect: 3
  Precision: 1.0,  Recall: 0.4,  F1: 0.5714285714285715
08IZY6G9.csv
  Correct: 15, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0CZW7M0F.csv
  Correct: 2, Incorrect: 1
  Precision: 1.0,  Recall: 0.6666666666666666,  F1: 0.8
0DYQDSCY.csv
  Correct: 0, Incorrect: 10
  Precision: 0,  Recall: 0.0,  F1: 0
0ER7T81U.csv
  Correct: 6, Incorrect: 2
  Precision: 1.0,  Recall: 0.75,  F1: 0.8571428571428571
0JGLUKC7.csv
  Correct: 2, Incorrect: 5
  Precision: 1.0,  Recall: 0.2857142857142857,  F1: 0.4444444444444445
0JLTIHOL.csv
  Correct: 0, Incorrect: 4
  Precision: 0,  Recall: 0.0,  F1: 0
0LBWW5L4.csv
  Correct: 0, Incorrect: 8
  Precision: 0,  Recall: 0.0,  F1: 0
0N0NEYH9.csv
  Correct: 6, Incorrect: 4
  Precision: 1.0,  Recall: 0.6,  F1: 0.7499999999999999
0N5SQTLA.csv
  Correct: 0, Incorrect: 8
  Precision: 0,  Recall: 0.0,  F1: 0
Average Precision: 0.6, Average Recall: 0.3702380952380952, Average F1: 0.4423015873015873


# CTA (Column-Type Annotation)

In [107]:
cta = pd.read_csv("./data/WikidataTables2024R1/DataSets/Valid/gt/cta_gt.csv", header=None)

In [108]:
cta[cta[0] == results[0][0].removesuffix('.csv')]

,0,1,2
392,03ENJ6XM,0,http://www.wikidata.org/entity/Q54050


In [109]:
results[0][1].df_classes

,col0,col1,col2
0,http://www.wikidata.org/entity/Q207326,NaN,NaN


In [110]:
avg_precision = 0
avg_recall = 0
avg_f1 = 0

for res_no in range(len(results)):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    subset_df = cta[cta[0] == results[res_no][0].removesuffix('.csv')]
    
    for idx, row in subset_df.iterrows():
        mapping = results[res_no][1].df_classes.iloc[0, row[1]]

        if mapping == row[2]:
            true_positives += 1
        else:
            if mapping != mapping:
                false_negatives += 1
            else:
                false_positives += 1

    print(f"{results[res_no][0]}\n  Correct: {true_positives}, Incorrect: {false_negatives + false_positives}")

    try:
        precision = true_positives / (true_positives + false_positives)
    except ZeroDivisionError:
        precision = 0
    avg_precision += precision

    try:
        recall = true_positives / (true_positives + false_negatives)
    except ZeroDivisionError:
        recall = 0
    avg_recall += recall

    try:
        f1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0
    avg_f1 += f1
    
    print(f"  Precision: {precision},  Recall: {recall},  F1: {f1}")

print(f"Average Precision: {avg_precision / len(results)}, Average Recall: {avg_recall / len(results)}, Average F1: {avg_f1 / len(results)}")

03ENJ6XM.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
08IZY6G9.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0CZW7M0F.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0DYQDSCY.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
0ER7T81U.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0JGLUKC7.csv
  Correct: 0, Incorrect: 2
  Precision: 0.0,  Recall: 0.0,  F1: 0
0JLTIHOL.csv
  Correct: 0, Incorrect: 2
  Precision: 0.0,  Recall: 0.0,  F1: 0
0LBWW5L4.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
0N0NEYH9.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0N5SQTLA.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
Average Precision: 0.4, Average Recall: 0.4, Average F1: 0.4


# CPA (Columns-Property Annotation)

In [111]:
cpa = pd.read_csv("./data/WikidataTables2024R1/DataSets/Valid/gt/cpa_gt.csv", header=None)

In [112]:
avg_precision = 0
avg_recall = 0
avg_f1 = 0

for res_no in range(len(results)):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    subset_df = cpa[cpa[0] == results[res_no][0].removesuffix('.csv')]
    
    for idx, row in subset_df.iterrows():
        mapping = results[res_no][1].df_relations.iloc[row[1], row[2]]
        
        if mapping == row[3]:
            true_positives += 1
        else:
            if mapping != mapping:
                false_negatives += 1
            else:
                false_positives += 1

    print(f"{results[res_no][0]}\n  Correct: {true_positives}, Incorrect: {false_negatives + false_positives}")

    try:
        precision = true_positives / (true_positives + false_positives)
    except ZeroDivisionError:
        precision = 0
    avg_precision += precision

    try:
        recall = true_positives / (true_positives + false_negatives)
    except ZeroDivisionError:
        recall = 0
    avg_recall += recall

    try:
        f1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0
    avg_f1 += f1
    
    print(f"  Precision: {precision},  Recall: {recall},  F1: {f1}")

print(f"Average Precision: {avg_precision / len(results)}, Average Recall: {avg_recall / len(results)}, Average F1: {avg_f1 / len(results)}")

03ENJ6XM.csv
  Correct: 2, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
08IZY6G9.csv
  Correct: 2, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0CZW7M0F.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0DYQDSCY.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
0ER7T81U.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0JGLUKC7.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0JLTIHOL.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
0LBWW5L4.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
0N0NEYH9.csv
  Correct: 1, Incorrect: 0
  Precision: 1.0,  Recall: 1.0,  F1: 1.0
0N5SQTLA.csv
  Correct: 0, Incorrect: 1
  Precision: 0.0,  Recall: 0,  F1: 0
Average Precision: 0.6, Average Recall: 0.6, Average F1: 0.6
